In [2]:
from ncpol2sdpa import*
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
 # Set Parameters
level = 1
group = 2
trajectory = [2,3]
sum_traj = sum(trajectory)

# Generate observations of multiple trajectories
data=pd.read_csv('FairOutput.csv',header=None)
Y=data.to_numpy()

In [68]:
def fairA(T):

    time_start = time.time()
    
    start=[0]+random.sample(range(0,3),trajectory[0]-1)+[0]+random.sample(range(0,2),trajectory[1]-1)
    end=random.sample(range(T-3,T),trajectory[0]-1)+[T]+random.sample(range(T-2,T),trajectory[1]-1)+[T]
    length=[end[i]-start[i] for i in range(sum_traj)]

    # Decision Variables
    G = generate_operators("G", n_vars=1, hermitian=False, commutative=False)[0]
    Fdash = generate_operators("Fdash", n_vars=1, hermitian=False, commutative=False)[0]
    z = generate_operators("z", n_vars=1, hermitian=False, commutative=False)[0]
    m = generate_operators("m", n_vars=T+1, hermitian=False, commutative=False)
    q = generate_operators("q", n_vars=T, hermitian=False, commutative=False)
    p = generate_operators("p", n_vars=T, hermitian=False, commutative=False)
    f = generate_operators("f", n_vars=T, hermitian=False, commutative=False)

# Constraints
    ine1 = [f[i] - Fdash*m[i+1] - p[i] for i in range(T)]
    ine2 = [-f[i] + Fdash*m[i+1] + p[i] for i in range(T)]
    ine3 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]
    max1 = [z-1/trajectory[0]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(0,trajectory[0]))]
    max2 = [z-1/trajectory[1]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(trajectory[0],sum_traj))]
    #max3 = [z-(Y[t,j]-f[t])**2 for j in range(sum_traj) for t in range(start[j],end[j]) ]

    ines_A = ine1+ine2+ine3+ine4+max1+max2
    #ines_B = ine1+ine2+ine3+ine4+max3


# Objective
    obj_A = z + 1*sum(p[i]**2 for i in range(T)) # 5 is optimal for level 1
    #obj_B = z + 3*sum(p[i]**2 for i in range(T)) # 5 is optimal for level 1

# Solve the fair NCPO A
    sdp_A = SdpRelaxation(variables = flatten([G,Fdash,z,f,p,m,q]))
    sdp_A.get_relaxation(level, objective=obj_A, inequalities=ines_A)
#sdp.get_relaxation(level, objective=obj, inequalities=ines,substitutions=subs)
    sdp_A.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:\\...\\sdpa7-windows\\sdpa.exe"})
    
    time_end = time.time()
    return time_end-time_start

In [69]:
def fairB(T):
    time_start = time.time()
    
    start=[0]+random.sample(range(0,3),trajectory[0]-1)+[0]+random.sample(range(0,2),trajectory[1]-1)
    end=random.sample(range(T-3,T),trajectory[0]-1)+[T]+random.sample(range(T-2,T),trajectory[1]-1)+[T]
    length=[end[i]-start[i] for i in range(sum_traj)]

    # Decision Variables
    G = generate_operators("G", n_vars=1, hermitian=False, commutative=False)[0]
    Fdash = generate_operators("Fdash", n_vars=1, hermitian=False, commutative=False)[0]
    z = generate_operators("z", n_vars=1, hermitian=False, commutative=False)[0]
    m = generate_operators("m", n_vars=T+1, hermitian=False, commutative=False)
    q = generate_operators("q", n_vars=T, hermitian=False, commutative=False)
    p = generate_operators("p", n_vars=T, hermitian=False, commutative=False)
    f = generate_operators("f", n_vars=T, hermitian=False, commutative=False)

# Constraints
    ine1 = [f[i] - Fdash*m[i+1] - p[i] for i in range(T)]
    ine2 = [-f[i] + Fdash*m[i+1] + p[i] for i in range(T)]
    ine3 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]
    #max1 = [z-1/trajectory[0]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(0,trajectory[0]))]
    #max2 = [z-1/trajectory[1]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(trajectory[0],sum_traj))]
    max3 = [z-(Y[t,j]-f[t])**2 for j in range(sum_traj) for t in range(start[j],end[j]) ]

    #ines_A = ine1+ine2+ine3+ine4+max1+max2
    ines_B = ine1+ine2+ine3+ine4+max3


# Objective
    #obj_A = z + 1*sum(p[i]**2 for i in range(T)) # 5 is optimal for level 1
    obj_B = z + 3*sum(p[i]**2 for i in range(T)) # 5 is optimal for level 1

# Solve the fair NCPO B
    sdp_B = SdpRelaxation(variables = flatten([G,Fdash,z,f,p,m,q]))
    sdp_B.get_relaxation(level, objective=obj_B, inequalities=ines_B)
#sdp.get_relaxation(level, objective=obj, inequalities=ines,substitutions=subs)
    sdp_B.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:\\Users\\zhouq\\Documents\\sdpa7-windows\\sdpa.exe"})
    
    time_end = time.time()
    return time_end-time_start

In [75]:
Amean=[]
Astd=[]
Bmean=[]
Bstd=[]
for t in range(5,21):
    Am=[]
    Bm=[]
    for r in range(3):
        Am.append(fairA(t))
        Bm.append(fairB(t))
    Amean.append(np.mean(Am))
    Astd.append(np.std(Am))
    Bmean.append(np.mean(Bm))
    Bstd.append(np.std(Bm))

In [81]:
A=pd.DataFrame(list(zip(Amean,Astd)))
B=pd.DataFrame(list(zip(Bmean,Bstd)))
A.to_csv (r'FairAtime0520_sdpa.csv', index = False, header=False)
B.to_csv (r'FairBtime0520_sdpa.csv', index = False, header=False)